In [ ]:
#hide
#all_slow

In [ ]:
#hide
!pip install voila
!jupyter serverextension enable --sys-prefix voila

# Steel Defect Detection

> Severstal Deep Learning segmentation case study.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from steel_segmentation.all import *
from fastai.vision.all import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, VBox
from IPython.display import Image

In [ ]:
def opt_func(params, **kwargs): return OptimWrapper(params, torch.optim.Adam, **kwargs)
def splitter(m): return convert_params([[m.encoder], [m.decoder], [m.segmentation_head]])

In [ ]:
learner = load_learner(models_dir/"exports"/"efficientnet-b2.pkl")

Upload a steel image here:

In [ ]:
#hide_output
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
rles, preds, probs = learner.predict(img)

In [ ]:
defects = list(preds.float().argmax(0).unique().numpy())[1:]
n = len(defects)
defects_word = "defects" if n!=1 else "defect"
types_word = "types" if n!=1 else "type"
if n > 0:
    title = f"Predicted n°{n} {defects_word} of {types_word}: {' '.join([str(o) for o in defects])}"
else:
    title = f"Predicted n°0 {defects_word}"

In [ ]:
#hide_output
lbl_pred = widgets.Label()
lbl_pred.value = title
lbl_pred

Label(value='Predicted n°1 defect of type: 2')

In [ ]:
#hide_output
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: plot_mask_image(title, np.array(img), preds.permute(1,2,0).float().numpy())
out_pl

Output()

In [ ]:
#hide_output
btn_run = widgets.Button(description='Classify')
btn_run

Button(description='Classify', style=ButtonStyle())

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    
    rles, preds, probs = learner.predict(img)
        
    defects = list(preds.float().argmax(0).unique().numpy())[1:]
    n = len(defects)
    defects_word = "defects" if n!=1 else "defect"
    types_word = "types" if n!=1 else "type"
    if n > 0:
        title = f"Predicted n°{n} {defects_word} of {types_word}: {' '.join([str(o+1) for o in defects])}"
    else:
        title = f"Predicted n°0 {defects_word}"
        
    out_pl.clear_output()
    with out_pl: 
        plot_mask_image(title, np.array(img), preds.permute(1,2,0).float().numpy())
    lbl_pred.value = title
        
btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Detect steel defects with image segmentation'), 
      btn_upload, btn_run, out_pl, lbl_pred])